[![nbviewer](https://raw.githubusercontent.com/jupyter/design/master/logos/Badges/nbviewer_badge.svg)](https://nbviewer.jupyter.org/github/elastic/geneve/blob/main/tests/reports/documents_from_queries.ipynb)
[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/elastic/geneve/main?labpath=tests/reports/documents_from_queries.ipynb)

# Documents generation from test queries

This Jupyter Notebook captures the unit test results of documents generation from queries.
Here you can learn what kind of queries the emitter handles and the documents it generates.

To edit an input cell, just click in its gray area. To execute it, press `Ctrl+Enter`.

Curious about the inner workings? Read [here](../../docs/events_generation.md).
Need help in using a Jupyter Notebook?
Read [here](https://jupyter-notebook.readthedocs.io/en/stable/notebook.html#structure-of-a-notebook-document).

## Table of contents
   1. [Preliminaries](#Preliminaries)
   1. [Mono-branch mono-document](#Mono-branch-mono-document)
   1. [Multi-branch mono-document](#Multi-branch-mono-document)
   1. [Mono-branch multi-document](#Mono-branch-multi-document)
   1. [Multi-branch multi-document](#Multi-branch-multi-document)
   1. [Error conditions](#Error-conditions)
   1. [Cardinality](#Cardinality)
   1. [Any oddities?](#Any-oddities?)

## Preliminaries

This is an auxiliary cell, it prepares the environment for the rest of this notebook.

In [ ]:
import os; os.chdir('../..')  # use the repo's root as base for local modules import
from geneve.events_emitter import SourceEvents
from geneve.utils import load_schema

# load the ECS schema
SourceEvents.schema = load_schema('./etc/ecs-8.2.0.tar.gz', 'generated/ecs/ecs_flat.yml')

def emit(query, timestamp=False, complete=True, count=1):
    try:
        events = SourceEvents.from_query(query).emit(timestamp=timestamp, complete=complete, count=count)
        if complete:
            return [[event.doc for event in branch] for branch in events]
        else:
            return [event.doc for event in events]
    except Exception as e:
        print(e)

## How to read the test results

If you opened this as freshly generated, the output cells content comes from the unit tests run and
you can read it as a plain test report. Such content is generated in a controlled environment and is
meant not to change between unit tests runs.
The notebook itself does not run in such controlled environment therefore executing these cells, even
if unmodified, will likely lead to different results each time.

On the other hand, you can experiment and modify the queries in the input cells, check the results
and, why not?, report any interesting finding. You can also add and remove cells at will.

## Mono-branch mono-document

What follows are queries that shall trigger a signal with just a single source event,
therefore at most one document is generated for each execution.

In [ ]:
emit('''
    any where true
''')

[[{}]]

In [ ]:
emit('''
    any where not false
''')

[[{}]]

In [ ]:
emit('''
    any where not (true and false)
''')

[[{}]]

In [ ]:
emit('''
    any where not (false or false)
''')

[[{}]]

In [ ]:
emit('''
    network where source.port > 512 and source.port < 1024
''')

[[{'event': {'category': ['network']}, 'source': {'port': 794}}]]

In [ ]:
emit('''
    network where not (source.port < 512 or source.port > 1024)
''')

[[{'event': {'category': ['network']}, 'source': {'port': 1021}}]]

In [ ]:
emit('''
    network where destination.port not in (80, 443)
''')

[[{'event': {'category': ['network']}, 'destination': {'port': 7564}}]]

In [ ]:
emit('''
    network where not destination.port in (80, 443)
''')

[[{'event': {'category': ['network']}, 'destination': {'port': 246}}]]

In [ ]:
emit('''
    network where destination.port == 22 and destination.port in (80, 443) or destination.port == 25
''')

[[{'event': {'category': ['network']}, 'destination': {'port': 25}}]]

In [ ]:
emit('''
    process where process.name == "regsvr32.exe"
''')

[[{'event': {'category': ['process']}, 'process': {'name': 'regsvr32.exe'}}]]

In [ ]:
emit('''
    process where process.name != "regsvr32.exe"
''')

[[{'event': {'category': ['process']}, 'process': {'name': 'Bmc'}}]]

In [ ]:
emit('''
    process where process.pid != 0
''')

[[{'event': {'category': ['process']}, 'process': {'pid': 3009213395}}]]

In [ ]:
emit('''
    process where process.pid >= 0
''')

[[{'event': {'category': ['process']}, 'process': {'pid': 1706296503}}]]

In [ ]:
emit('''
    process where process.pid > 0
''')

[[{'event': {'category': ['process']}, 'process': {'pid': 2505219495}}]]

In [ ]:
emit('''
    process where process.code_signature.exists == true
''')

[[{'event': {'category': ['process']}, 'process': {'code_signature': {'exists': True}}}]]

In [ ]:
emit('''
    process where process.code_signature.exists != true
''')

[[{'event': {'category': ['process']}, 'process': {'code_signature': {'exists': False}}}]]

In [ ]:
emit('''
    any where network.protocol == "some protocol"
''')

[[{'network': {'protocol': 'some protocol'}}]]

In [ ]:
emit('''
    any where process.pid == null
''')

[[{}]]

In [ ]:
emit('''
    any where not process.pid != null
''')

[[{}]]

In [ ]:
emit('''
    any where process.pid != null
''')

[[{'process': {'pid': 102799507}}]]

In [ ]:
emit('''
    any where not process.pid == null
''')

[[{'process': {'pid': 2584819203}}]]

In [ ]:
emit('''
    process where process.name == "regsvr32.exe" and process.parent.name == "cmd.exe"
''')

[[{'event': {'category': ['process']}, 'process': {'name': 'regsvr32.exe', 'parent': {'name': 'cmd.exe'}}}]]

In [ ]:
emit('''
    process where process.name : ("*.EXE", "*.DLL")
''')

[[{'event': {'category': ['process']}, 'process': {'name': 'LeneQZk.EXE'}}]]

In [ ]:
emit('''
    process where process.args != null
''')

[[{'event': {'category': ['process']}, 'process': {'args': ['CJI']}}]]

In [ ]:
emit('''
    process where process.args : "-f" and process.args == "-r"
''')

[[{'event': {'category': ['process']}, 'process': {'args': ['-f', '-r']}}]]

In [ ]:
emit('''
    network where destination.ip == "127.0.0.1"
''')

[[{'event': {'category': ['network']}, 'destination': {'ip': '127.0.0.1'}}]]

In [ ]:
emit('''
    network where cidrMatch(destination.ip, "10.0.0.0/8", "192.168.0.0/16")
''')

[[{'event': {'category': ['network']}, 'destination': {'ip': '10.77.153.19'}}]]

In [ ]:
emit('''
    network where not cidrMatch(destination.ip, "10.0.0.0/8", "192.168.0.0/16")
''')

[[{'event': {'category': ['network']}, 'destination': {'ip': '0.225.250.37'}}]]

In [ ]:
emit('''
    network where destination.ip != null
''')

[[{'event': {'category': ['network']}, 'destination': {'ip': '143.174.17.137'}}]]

In [ ]:
emit('''
    network where destination.ip == "::1"
''')

[[{'event': {'category': ['network']}, 'destination': {'ip': '::1'}}]]

In [ ]:
emit('''
    network where destination.ip == "822e::/16"
''')

[[{'event': {'category': ['network']}, 'destination': {'ip': '822e:f740:dcc5:503a:946f:261:2c07:f7a5'}}]]

In [ ]:
emit('''
    event.category:network and destination.ip:"822e::/16"
''')

[[{'event': {'category': ['network']}, 'destination': {'ip': '822e:f477:4aa3:d9c5:7494:c408:2f13:daeb'}}]]

In [ ]:
emit('''
    network where host.ip != null
''')

[[{'event': {'category': ['network']}, 'host': {'ip': ['238.136.72.63']}}]]

In [ ]:
emit('''
    event.category:network and host.ip:"822e::/96"
''')

[[{'event': {'category': ['network']}, 'host': {'ip': ['822e::c0f6:4909']}}]]

## Multi-branch mono-document

Following queries have one or more disjunctive operators (eg. _or_) which split the query
in multiple _branches_. Each branch shall generate a single source event.

In [ ]:
emit('''
    network where not (source.port > 512 and source.port < 1024)
''')

[[{'event': {'category': ['network']}, 'source': {'port': 182}}],
 [{'event': {'category': ['network']}, 'source': {'port': 54422}}]]

In [ ]:
emit('''
    network where source.port > 512 or source.port < 1024
''')

[[{'event': {'category': ['network']}, 'source': {'port': 44925}}],
 [{'event': {'category': ['network']}, 'source': {'port': 516}}]]

In [ ]:
emit('''
    network where source.port < 2000 and (source.port > 512 or source.port > 1024)
''')

[[{'event': {'category': ['network']}, 'source': {'port': 1334}}],
 [{'event': {'category': ['network']}, 'source': {'port': 1034}}]]

In [ ]:
emit('''
    network where (source.port > 512 or source.port > 1024) and source.port < 2000
''')

[[{'event': {'category': ['network']}, 'source': {'port': 575}}],
 [{'event': {'category': ['network']}, 'source': {'port': 1158}}]]

In [ ]:
emit('''
    network where (source.port > 1024 or source.port < 2000) and (source.port < 4000 or source.port > 512)
''')

[[{'event': {'category': ['network']}, 'source': {'port': 1970}}],
 [{'event': {'category': ['network']}, 'source': {'port': 52226}}],
 [{'event': {'category': ['network']}, 'source': {'port': 692}}],
 [{'event': {'category': ['network']}, 'source': {'port': 1464}}]]

In [ ]:
emit('''
    network where destination.port in (80, 443)
''')

[[{'event': {'category': ['network']}, 'destination': {'port': 80}}],
 [{'event': {'category': ['network']}, 'destination': {'port': 443}}]]

In [ ]:
emit('''
    process where process.name == "regsvr32.exe" or process.parent.name == "cmd.exe"
''')

[[{'event': {'category': ['process']}, 'process': {'name': 'regsvr32.exe'}}],
 [{'event': {'category': ['process']}, 'process': {'parent': {'name': 'cmd.exe'}}}]]

In [ ]:
emit('''
    process where process.name == "regsvr32.exe" or process.name == "cmd.exe" or process.name == "powershell.exe"
''')

[[{'event': {'category': ['process']}, 'process': {'name': 'regsvr32.exe'}}],
 [{'event': {'category': ['process']}, 'process': {'name': 'cmd.exe'}}],
 [{'event': {'category': ['process']}, 'process': {'name': 'powershell.exe'}}]]

In [ ]:
emit('''
    process where process.name in ("regsvr32.exe", "cmd.exe", "powershell.exe")
''')

[[{'event': {'category': ['process']}, 'process': {'name': 'regsvr32.exe'}}],
 [{'event': {'category': ['process']}, 'process': {'name': 'cmd.exe'}}],
 [{'event': {'category': ['process']}, 'process': {'name': 'powershell.exe'}}]]

In [ ]:
emit('''
    process where process.name in ("regsvr32.exe", "cmd.exe") or process.name == "powershell.exe"
''')

[[{'event': {'category': ['process']}, 'process': {'name': 'regsvr32.exe'}}],
 [{'event': {'category': ['process']}, 'process': {'name': 'cmd.exe'}}],
 [{'event': {'category': ['process']}, 'process': {'name': 'powershell.exe'}}]]

In [ ]:
emit('''
    process where event.type in ("start", "process_started") and process.args : "dump-keychain" and process.args : "-d"
''')

[[{'event': {'category': ['process'], 'type': ['start']}, 'process': {'args': ['dump-keychain', '-d']}}],
 [{'event': {'category': ['process'], 'type': ['process_started']}, 'process': {'args': ['dump-keychain', '-d']}}]]

In [ ]:
emit('''
    event.type:(start or process_started) and (process.args:"dump-keychain" and process.args:"-d")
''')

[[{'event': {'type': ['start']}, 'process': {'args': ['dump-keychain', '-d']}}],
 [{'event': {'type': ['process_started']}, 'process': {'args': ['dump-keychain', '-d']}}]]

## Mono-branch multi-document

Following queries instead require multiple related source events, it's not analyzed only each
event content but also the relation with each others. Therefore a senquence of documents is generated
each time and all the documents in the sequence are required for one signal to be generated.

In [ ]:
emit('''
    sequence
        [process where process.name : "cmd.exe"]
        [process where process.parent.name : "cmd.exe"]
''')

[[{'event': {'category': ['process']}, 'process': {'name': 'cmd.exe'}},
  {'event': {'category': ['process']}, 'process': {'parent': {'name': 'cmd.exe'}}}]]

In [ ]:
emit('''
    sequence by user.id
        [process where process.name : "cmd.exe"]
        [process where process.parent.name : "cmd.exe"]
''')

[[{'event': {'category': ['process']}, 'process': {'name': 'cmd.exe'}, 'user': {'id': 'klM'}},
  {'event': {'category': ['process']}, 'process': {'parent': {'name': 'cmd.exe'}}, 'user': {'id': 'klM'}}]]

In [ ]:
emit('''
    sequence
        [process where process.name : "cmd.exe"] by user.id
        [process where process.parent.name : "cmd.exe"] by user.name
''')

[[{'event': {'category': ['process']}, 'process': {'name': 'cmd.exe'}, 'user': {'id': 'fmC'}},
  {'event': {'category': ['process']}, 'process': {'parent': {'name': 'cmd.exe'}}, 'user': {'name': 'fmC'}}]]

## Multi-branch multi-document

Same as above but one or more queries in the sequence have a disjunction (eg. _or_ operator) therefore
multiple sequences shall be generated.

In [ ]:
emit('''
    sequence
        [process where process.name : "cmd.exe"]
        [process where process.parent.name : "cmd.exe" or process.name : "powershell.exe"]
''')

[[{'event': {'category': ['process']}, 'process': {'name': 'cmd.exe'}},
  {'event': {'category': ['process']}, 'process': {'parent': {'name': 'cmd.exe'}}}],
 [{'event': {'category': ['process']}, 'process': {'name': 'cmd.exe'}},
  {'event': {'category': ['process']}, 'process': {'name': 'powershell.exe'}}]]

In [ ]:
emit('''
    sequence by user.id
        [process where process.name : "cmd.exe"]
        [process where process.parent.name : "cmd.exe" or process.name : "powershell.exe"]
''')

[[{'event': {'category': ['process']}, 'process': {'name': 'cmd.exe'}, 'user': {'id': 'pKP'}},
  {'event': {'category': ['process']}, 'process': {'parent': {'name': 'cmd.exe'}}, 'user': {'id': 'pKP'}}],
 [{'event': {'category': ['process']}, 'process': {'name': 'cmd.exe'}, 'user': {'id': 'dYR'}},
  {'event': {'category': ['process']}, 'process': {'name': 'powershell.exe'}, 'user': {'id': 'dYR'}}]]

In [ ]:
emit('''
    sequence
        [process where process.name in ("cmd.exe", "powershell.exe")] by process.name
        [process where process.name in ("cmd.exe", "powershell.exe")] by process.parent.name
''')

[[{'event': {'category': ['process']}, 'process': {'name': 'cmd.exe'}},
  {'event': {'category': ['process']}, 'process': {'name': 'cmd.exe', 'parent': {'name': 'cmd.exe'}}}],
 [{'event': {'category': ['process']}, 'process': {'name': 'cmd.exe'}},
  {'event': {'category': ['process']}, 'process': {'name': 'powershell.exe', 'parent': {'name': 'cmd.exe'}}}],
 [{'event': {'category': ['process']}, 'process': {'name': 'powershell.exe'}},
  {'event': {'category': ['process']}, 'process': {'name': 'cmd.exe', 'parent': {'name': 'powershell.exe'}}}],
 [{'event': {'category': ['process']}, 'process': {'name': 'powershell.exe'}},
  {'event': {'category': ['process']}, 'process': {'name': 'powershell.exe', 'parent': {'name': 'powershell.exe'}}}]]

In [ ]:
emit('''
    sequence by user.id
        [process where process.name in ("cmd.exe", "powershell.exe")] by process.name
        [process where process.name in ("cmd.exe", "powershell.exe")] by process.parent.name
''')

[[{'event': {'category': ['process']}, 'process': {'name': 'cmd.exe'}, 'user': {'id': 'aPd'}},
  {'event': {'category': ['process']}, 'process': {'name': 'cmd.exe', 'parent': {'name': 'cmd.exe'}}, 'user': {'id': 'aPd'}}],
 [{'event': {'category': ['process']}, 'process': {'name': 'cmd.exe'}, 'user': {'id': 'aiW'}},
  {'event': {'category': ['process']}, 'process': {'name': 'powershell.exe', 'parent': {'name': 'cmd.exe'}}, 'user': {'id': 'aiW'}}],
 [{'event': {'category': ['process']}, 'process': {'name': 'powershell.exe'}, 'user': {'id': 'tSw'}},
  {'event': {'category': ['process']}, 'process': {'name': 'cmd.exe', 'parent': {'name': 'powershell.exe'}}, 'user': {'id': 'tSw'}}],
 [{'event': {'category': ['process']}, 'process': {'name': 'powershell.exe'}, 'user': {'id': 'JEL'}},
  {'event': {'category': ['process']}, 'process': {'name': 'powershell.exe', 'parent': {'name': 'powershell.exe'}}, 'user': {'id': 'JEL'}}]]

## Error conditions

Not all the queries make sense, no documents can be generated for those that cannot logically be ever
matched. In such cases an error is reported, as the following cells show.

Here you can challenge the generation engine first hand and check that all the due errors are reported
and make sense to you.

In [ ]:
emit('''
    any where false
''')

Root without branches

In [ ]:
emit('''
    any where not true
''')

Root without branches

In [ ]:
emit('''
    any where not (true and true)
''')

Root without branches

In [ ]:
emit('''
    any where not (true or false)
''')

Root without branches

In [ ]:
emit('''
    any where process.pid == null and process.pid != null
''')

Unsolvable constraints: process.pid (cannot be non-null)

In [ ]:
emit('''
    any where process.pid > 0 and process.pid == null
''')

Unsolvable constraints: process.pid (cannot be null)

In [ ]:
emit('''
    any where process.name != null and process.name == null
''')

Unsolvable constraints: process.name (cannot be null)

In [ ]:
emit('''
    any where process.name == "cmd.exe" and process.name == null
''')

Unsolvable constraints: process.name (cannot be null)

In [ ]:
emit('''
    process where process.pid == 0
''')

Unsolvable constraints: process.pid (out of boundary, 1 <= 0 <= 4294967295)

In [ ]:
emit('''
    process where process.pid <= 0
''')

Unsolvable constraints: process.pid (empty solution space, 1 <= x <= 0)

In [ ]:
emit('''
    process where process.pid < 0
''')

Unsolvable constraints: process.pid (empty solution space, 1 <= x <= -1)

In [ ]:
emit('''
    any where network.protocol == "http" and network.protocol == "https"
''')

Unsolvable constraints ==: network.protocol (is already 'http', cannot set to 'https')

In [ ]:
emit('''
    network where destination.port == 22 and destination.port in (80, 443)
''')

Root without branches

In [ ]:
emit('''
    network where not (source.port > 512 or source.port < 1024)
''')

Unsolvable constraints: source.port (empty solution space, 1024 <= x <= 512)

In [ ]:
emit('''
    sequence by process.name
        [process where process.name : "cmd.exe"]
        [process where process.name : "powershell.exe"]
''')

Unsolvable constraints ==: process.name (is already 'powershell.exe', cannot set to 'cmd.exe')

In [ ]:
emit('''
    sequence
        [process where process.name : "cmd.exe"] by process.name
        [process where process.parent.name : "powershell.exe"] by process.parent.name
''')

Unsolvable constraints ==: process.parent.name (is already 'powershell.exe', cannot set to 'cmd.exe')

In [ ]:
emit('''
    sequence by process.name
        [process where process.name == null]
        [process where process.name : "powershell.exe"]
''')

Unsolvable constraints: process.name (cannot be non-null)

## Cardinality

Cardinality constraints set an upper bound to the number of different values generated for a given field.

In [ ]:
emit('''
    process where process.pid > 0 and process.pid < 100 and _cardinality(process.pid, 0)
''', count=7)

[[{'event': {'category': ['process']}, 'process': {'pid': 35}}],
 [{'event': {'category': ['process']}, 'process': {'pid': 64}}],
 [{'event': {'category': ['process']}, 'process': {'pid': 30}}],
 [{'event': {'category': ['process']}, 'process': {'pid': 99}}],
 [{'event': {'category': ['process']}, 'process': {'pid': 85}}],
 [{'event': {'category': ['process']}, 'process': {'pid': 42}}],
 [{'event': {'category': ['process']}, 'process': {'pid': 95}}]]

In [ ]:
emit('''
    process where process.pid > 0 and process.pid < 100 and _cardinality(process.pid, 1)
''', count=7)

[[{'event': {'category': ['process']}, 'process': {'pid': 38}}],
 [{'event': {'category': ['process']}, 'process': {'pid': 38}}],
 [{'event': {'category': ['process']}, 'process': {'pid': 38}}],
 [{'event': {'category': ['process']}, 'process': {'pid': 38}}],
 [{'event': {'category': ['process']}, 'process': {'pid': 38}}],
 [{'event': {'category': ['process']}, 'process': {'pid': 38}}],
 [{'event': {'category': ['process']}, 'process': {'pid': 38}}]]

In [ ]:
emit('''
    process where process.pid > 0 and process.pid < 100 and _cardinality(process.pid, 2)
''', count=8)

[[{'event': {'category': ['process']}, 'process': {'pid': 38}}],
 [{'event': {'category': ['process']}, 'process': {'pid': 38}}],
 [{'event': {'category': ['process']}, 'process': {'pid': 38}}],
 [{'event': {'category': ['process']}, 'process': {'pid': 87}}],
 [{'event': {'category': ['process']}, 'process': {'pid': 87}}],
 [{'event': {'category': ['process']}, 'process': {'pid': 38}}],
 [{'event': {'category': ['process']}, 'process': {'pid': 38}}],
 [{'event': {'category': ['process']}, 'process': {'pid': 38}}]]

In [ ]:
emit('''
    network where source.ip == "10.0.0.0/24" and _cardinality(source.ip, 2)
''', count=7)

[[{'event': {'category': ['network']}, 'source': {'ip': '10.0.0.99'}}],
 [{'event': {'category': ['network']}, 'source': {'ip': '10.0.0.99'}}],
 [{'event': {'category': ['network']}, 'source': {'ip': '10.0.0.214'}}],
 [{'event': {'category': ['network']}, 'source': {'ip': '10.0.0.99'}}],
 [{'event': {'category': ['network']}, 'source': {'ip': '10.0.0.214'}}],
 [{'event': {'category': ['network']}, 'source': {'ip': '10.0.0.99'}}],
 [{'event': {'category': ['network']}, 'source': {'ip': '10.0.0.214'}}]]

In [ ]:
emit('''
    network where destination.ip == "1::/112" and _cardinality(destination.ip, 3)
''', count=7)

[[{'event': {'category': ['network']}, 'destination': {'ip': '1::1cf0'}}],
 [{'event': {'category': ['network']}, 'destination': {'ip': '1::f14'}}],
 [{'event': {'category': ['network']}, 'destination': {'ip': '1::f09b'}}],
 [{'event': {'category': ['network']}, 'destination': {'ip': '1::1cf0'}}],
 [{'event': {'category': ['network']}, 'destination': {'ip': '1::f09b'}}],
 [{'event': {'category': ['network']}, 'destination': {'ip': '1::1cf0'}}],
 [{'event': {'category': ['network']}, 'destination': {'ip': '1::1cf0'}}]]

In [ ]:
emit('''
    process where _cardinality(process.name, 3)
''', count=7)

[[{'event': {'category': ['process']}, 'process': {'name': 'tQY'}}],
 [{'event': {'category': ['process']}, 'process': {'name': 'FmB'}}],
 [{'event': {'category': ['process']}, 'process': {'name': 'Iwi'}}],
 [{'event': {'category': ['process']}, 'process': {'name': 'Iwi'}}],
 [{'event': {'category': ['process']}, 'process': {'name': 'FmB'}}],
 [{'event': {'category': ['process']}, 'process': {'name': 'tQY'}}],
 [{'event': {'category': ['process']}, 'process': {'name': 'tQY'}}]]

In [ ]:
emit('''
    network where destination.port in (22, 443) and _cardinality(destination.ip, 1)
''', count=4)

[[{'event': {'category': ['network']}, 'destination': {'ip': '1::f09b', 'port': 22}}],
 [{'event': {'category': ['network']}, 'destination': {'ip': '1::f09b', 'port': 443}}],
 [{'event': {'category': ['network']}, 'destination': {'ip': '1::f09b', 'port': 22}}],
 [{'event': {'category': ['network']}, 'destination': {'ip': '1::f09b', 'port': 443}}]]

In [ ]:
emit('''
    network where destination.port in (22, 443) and _cardinality(destination.ip, 2)
''', count=10)

[[{'event': {'category': ['network']}, 'destination': {'ip': '1::1cf0', 'port': 22}}],
 [{'event': {'category': ['network']}, 'destination': {'ip': '1::1cf0', 'port': 443}}],
 [{'event': {'category': ['network']}, 'destination': {'ip': '1::1cf0', 'port': 22}}],
 [{'event': {'category': ['network']}, 'destination': {'ip': '1::f09b', 'port': 443}}],
 [{'event': {'category': ['network']}, 'destination': {'ip': '1::f09b', 'port': 22}}],
 [{'event': {'category': ['network']}, 'destination': {'ip': '1::1cf0', 'port': 443}}],
 [{'event': {'category': ['network']}, 'destination': {'ip': '1::1cf0', 'port': 22}}],
 [{'event': {'category': ['network']}, 'destination': {'ip': '1::1cf0', 'port': 443}}],
 [{'event': {'category': ['network']}, 'destination': {'ip': '1::f09b', 'port': 22}}],
 [{'event': {'category': ['network']}, 'destination': {'ip': '1::1cf0', 'port': 443}}]]

In [ ]:
emit('''
    network where destination.port in (22, 443) and _cardinality(destination.ip, 3)
''', count=12)

[[{'event': {'category': ['network']}, 'destination': {'ip': '1::1cf0', 'port': 22}}],
 [{'event': {'category': ['network']}, 'destination': {'ip': '1::f14', 'port': 443}}],
 [{'event': {'category': ['network']}, 'destination': {'ip': '1::f14', 'port': 22}}],
 [{'event': {'category': ['network']}, 'destination': {'ip': '1::f14', 'port': 443}}],
 [{'event': {'category': ['network']}, 'destination': {'ip': '1::f09b', 'port': 22}}],
 [{'event': {'category': ['network']}, 'destination': {'ip': '1::1cf0', 'port': 443}}],
 [{'event': {'category': ['network']}, 'destination': {'ip': '1::f09b', 'port': 22}}],
 [{'event': {'category': ['network']}, 'destination': {'ip': '1::f14', 'port': 443}}],
 [{'event': {'category': ['network']}, 'destination': {'ip': '1::1cf0', 'port': 22}}],
 [{'event': {'category': ['network']}, 'destination': {'ip': '1::f09b', 'port': 443}}],
 [{'event': {'category': ['network']}, 'destination': {'ip': '1::f14', 'port': 22}}],
 [{'event': {'category': ['network']}, 'des

In [ ]:
emit('''
    sequence
            [network where _cardinality(destination.ip, 1) and destination.port < 1024] by destination.ip, destination.port
            [network where _cardinality(destination.ip, 2) and destination.port > 1024] by source.ip, source.port
''', count=5)

[[{'event': {'category': ['network']}, 'destination': {'ip': '1::f09b', 'port': 265}},
  {'event': {'category': ['network']}, 'source': {'ip': '1::f09b', 'port': 265}, 'destination': {'ip': '1::1cf0', 'port': 22623}}],
 [{'event': {'category': ['network']}, 'destination': {'ip': '1::f09b', 'port': 691}},
  {'event': {'category': ['network']}, 'source': {'ip': '1::f09b', 'port': 691}, 'destination': {'ip': '1::1cf0', 'port': 57576}}],
 [{'event': {'category': ['network']}, 'destination': {'ip': '1::f09b', 'port': 173}},
  {'event': {'category': ['network']}, 'source': {'ip': '1::f09b', 'port': 173}, 'destination': {'ip': '1::1cf0', 'port': 3897}}],
 [{'event': {'category': ['network']}, 'destination': {'ip': '1::f09b', 'port': 685}},
  {'event': {'category': ['network']}, 'source': {'ip': '1::f09b', 'port': 685}, 'destination': {'ip': '1::f09b', 'port': 58442}}],
 [{'event': {'category': ['network']}, 'destination': {'ip': '1::f09b', 'port': 929}},
  {'event': {'category': ['network']},

## Any oddities?

Did you find anything odd reviewing the report or playing with the documents emitter?
We are interested to know, feel free to [create an issue](https://github.com/elastic/geneve/issues/new).